<a href="https://colab.research.google.com/github/Pheonix10101/PRCV_p_5/blob/main/Greek.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
Project 5: Recognition using Deep Networks

Author: Samruddhi Raut

This file contain following tasks

Task 3:Use the trained network as an embedding space for images of written symbols.

A:Create a greek symbol data set

B:Create a truncated model

C:Project the greek symbols into the embedding space

D:Compute distances in the embedding space

E:Create your own greek symbol data

"""
#Import_statement
import torch
from torchvision.io import read_image
from torch.utils.data import DataLoader
import os
import csv
import cv2
import numpy as np
import torch.nn.functional as F
from matplotlib import pyplot as plt
from PIL import Image, ImageOps
import CNN
import new_network


**Defining** Submodel

In [ ]:
torch.manual_seed(42)

CONV_FILTER_SIZE = [5, 10]
DROPOUT_RATE = [0.5,0.8 ]
KNN_INPUT_FILENAME = '/content/greek_hand write'


class Submodel(CNN.NeuralNetwork):
    def __init__(self, conv_filter = 5, dropout_rate = 0.5):
        super().__init__(conv_filter, dropout_rate)

    def forward(self, x):# override the forward method
        x = self.conv1(x)# A convolution layer with 10 5x5 filters
        x = F.relu(F.max_pool2d(x, (2, 2)))# A max pooling layer with a 2x2 window and a ReLU function applied
        x = self.conv2(x)# A convolution layer with 20 5x5 filters
        x = self.conv2_drop(x)# A dropout layer with a 0.5 dropout rate (50%)
        x = F.relu(F.max_pool2d(x, (2, 2)))# A max pooling layer with a 2x2 window and a ReLU function applied
        x = F.relu(self.fc1(self.flat1(x)))# A flattening operation followed by a fully connected Linear layer with 50 nodes and a ReLU function on the
        return x

**Defining SSD**

In [ ]:

def ssd(a, b):
    d = np.sum((a - b) ** 2)
    return d

**Saving files to CSV**

saving intensity files
saving labels file

In [ ]:
def save_to_csv():
   
    #Save the intensity values and their labels into two csv files
    
    #intensity values
    with open('/content/G_intensity.csv', 'w', encoding = 'UTF8', newline = '') as f:
        writer = csv.writer(f)
        header = ['Filename', 'Intensity']
        writer.writerow(header)
        for filename in os.listdir('/content/greek-1'):
            image = Image.open(os.path.join('/content/greek-1', filename))
            writer.writerow([filename, np.array(image)])

    #label values
    with open('/content/G_labels.csv', 'w', encoding = 'UTF8', newline = '') as f:
        writer = csv.writer(f)
        header = ['Filename', 'Label']
        writer.writerow(header)
        for filename in os.listdir('/content/greek-1'):
            if 'alpha' in filename:
                writer.writerow([filename, 0])
            elif 'beta' in filename:
                writer.writerow([filename, 1])
            elif 'gamma' in filename:
                writer.writerow([filename, 2])
            elif 'eta' in filename:
                writer.writerow([filename, 3])
            elif 'phi' in filename:
                writer.writerow([filename, 4])

    dataset_greek_values = new_network.NumDraw_Data(annotations_file = '/content/G_labels.csv',
                                                   img_dir = '/content/greek-1')
    dataloader_greek_values = DataLoader(dataset = dataset_greek_values,
                                  batch_size = CNN.BATCH_SIZE_TEST,
                                  shuffle = False,
                                  num_workers = 4)

    return dataset_greek_values, dataloader_greek_values

**Apply the truncated network to the greek symbols (read from the CSV file) to get a set of 27 50 element vectors**

param greek_submodel:      submodel build from loaded network
.param dataloader_greek_values:    dataloader of greek dataset
return  overall shape of outputs, and labels
  

In [ ]:
def embedding(greek_submodel, dataloader_greek_values):
  
    greek_submodel.eval()
    results = []
    targets = []
    b = 0
    for data, target in dataloader_greek_values:
        output = greek_submodel(data)
        print("\nBatch %d:" % b)
        print("Input batch size: ", end = "")
        print(data.shape)
        print("Apply the submodel with 50-node dense layer to the data, "
              "we have the returned output with the shape of: ", end = "")
        print(output.shape)
        b += 1
# make sure no matter what the batch size is, the results will always keep all outputs, in this case,
# shape of 27 * 50; and targets will have the corresponding labels
        for i in range(len(output)):
            results.append(output[i])
            targets.append(target[i])
    print("\nShape of the output nodes from the submodel: ", end = "")
    print(torch.stack(results).shape)
    print("Number of the labels: ", end = "")
    print(torch.stack(targets).shape)

    return results, targets

**Extract 3 greek symbols**

param results: outputs from submodel
,param targets: labels corresponding to the results
,return selected alpha, beta, and gamma

In [ ]:
def get_greek_letter(results, targets):
    alpha = []
    beta = []
    gamma = []
    label = 99
    for i in range(len(results)):
        if targets[i].detach().numpy() != label and targets[i].detach().numpy() == 0:
            for j in results[i].detach().numpy():
                alpha.append(j)
            label = targets[i].detach().numpy()
        elif targets[i].detach().numpy() != label and targets[i].detach().numpy() == 1:
            for j in results[i].detach().numpy():
                beta.append(j)
            label = targets[i].detach().numpy()
        elif targets[i].detach().numpy() != label and targets[i].detach().numpy() == 2:
            for j in results[i].detach().numpy():
                gamma.append(j)
            label = targets[i].detach().numpy()

    return alpha, beta, gamma

**Calculate the label of the given data by assigning it the label of knn's label**
param k:       k nearest neighbors
param results: outputs from submodel
param targets: labels corresponding to the results
param a:       given example

In [ ]:
def knn(k, results, targets, a):
   
    sum_alpha = []
    sum_beta = []
    sum_gamma = []
    sum_eta = []
    sum_phi = []
    for i in range(len(results)):
        d = ssd(np.array(a), results[i].detach().numpy())
        print("%.2f" % d, end = " ")
        if targets[i].detach().numpy() == 0:
            sum_alpha.append(d)
        elif targets[i].detach().numpy() == 1:
            sum_beta.append(d)
        elif targets[i].detach().numpy() == 2:
            sum_gamma.append(d)
        elif targets[i].detach().numpy() == 3:
            sum_eta.append(d)
        elif targets[i].detach().numpy() == 4:
            sum_phi.append(d)
    sum_alpha.sort()
    sum_beta.sort()
    sum_gamma.sort()
    sum_eta.sort()
    sum_phi.sort()
    print("\nTotal distance to top kth label alpha: %.2f" % sum(sum_alpha[: k]))
    print("Total distance to top kth label beta: %.2f" % sum(sum_beta[: k]))
    print("Total distance to top kth label gamma: %.2f" % sum(sum_gamma[: k]))
    print("Total distance to top kth label eta: %.2f" % sum(sum_eta[: k]))
    print("Total distance to top kth label phi: %.2f" % sum(sum_phi[: k]))

    prediction = min(sum(sum_alpha[: k]), sum(sum_beta[: k]),
               sum(sum_gamma[: k]), sum(sum_eta[: k]),
               sum(sum_phi[: k]))

    if prediction == sum(sum_alpha[: k]):
        return "alpha"
    elif prediction == sum(sum_beta[: k]):
        return "beta"
    elif prediction == sum(sum_gamma[: k]):
        return "gamma"
    elif prediction == sum(sum_eta[: k]):
        return "eta"
    elif prediction == sum(sum_phi[: k]):
        return "phi"
    else:
        return "unknown"


Calculate the average distance from a given example to all of other examples
param results: outputs from submodel,param targets: labels corresponding to the results,param a:given example
  

In [ ]:
def calculate_avg(results, targets, a):
    
    avg_alpha = []
    avg_beta = []
    avg_gamma = []
    avg_eta = []
    avg_phi = []
    for i in range(len(results)):
        d = ssd(np.array(a), results[i].detach().numpy())
        print("%.2f" % d, end = " ")
        if d == 0:
            continue
        if targets[i].detach().numpy() == 0:
            avg_alpha.append(d)
        elif targets[i].detach().numpy() == 1:
            avg_beta.append(d)
        elif targets[i].detach().numpy() == 2:
            avg_gamma.append(d)
        elif targets[i].detach().numpy() == 3:
            avg_eta.append(d)
        elif targets[i].detach().numpy() == 4:
            avg_phi.append(d)

    print("\nAverage distance to label alpha: %.2f" % (sum(avg_alpha) / len(avg_alpha)))
    print("Average distance to label beta: %.2f" % (sum(avg_beta) / len(avg_beta)))
    print("Average distance to label gamma: %.2f" % (sum(avg_gamma) / len(avg_gamma)))
    print("Average distance to label eta: %.2f" % (sum(avg_eta) / len(avg_eta)))
    print("Average distance to label phi: %.2f" % (sum(avg_phi) / len(avg_phi)))

    prediction = min((sum(avg_alpha) / len(avg_alpha)), (sum(avg_beta) / len(avg_beta)),
               (sum(avg_gamma) / len(avg_gamma)), (sum(avg_eta) / len(avg_eta)),
               (sum(avg_phi) / len(avg_phi)))

    if prediction == (sum(avg_alpha) / len(avg_alpha)):
        return "alpha"
    elif prediction == (sum(avg_beta) / len(avg_beta)):
        return "beta"
    elif prediction == (sum(avg_gamma) / len(avg_gamma)):
        return "gamma"
    elif prediction == (sum(avg_eta) / len(avg_eta)):
        return "eta"
    elif prediction == (sum(avg_phi) / len(avg_phi)):
        return "phi"
    else:
        return "unknown"

**Evaluating hand written greek letter* **

In [ ]:
def evaluate_greek_written(greek_submodel, results, targets, is_knn, s, dr):
    hand_write_greeks = new_network.NumDraw_Data(annotations_file = '/content/greek_hand write.csv',
                                                       img_dir = '/content/greek_hand write')
    hand_write_greeks_dataloader = DataLoader(dataset = hand_write_greeks,
                                              batch_size = CNN.BATCH_SIZE_TEST,
                                              shuffle = False,
                                              num_workers = 4)

    print("\nBuilding embedding space for hand write greeks")
    hand_write_results, hand_write_targets = embedding(greek_submodel, hand_write_greeks_dataloader)

    imgs = []
    for data, target in hand_write_greeks_dataloader:
        imgs.append(data)

    if (len(hand_write_results) / CNN.BATCH_SIZE_TEST).is_integer():
        batch_num = len(hand_write_results) / CNN.BATCH_SIZE_TEST
    else:
        batch_num = int(len(hand_write_results) / CNN.BATCH_SIZE_TEST) + 1

    predictions = []
    for i in range(len(hand_write_results)):
        print("\nDistance from selected image %d to others: " % i)
        if is_knn:
            predictions.append(knn(7, results, targets, hand_write_results[i].detach().numpy()))
        else:
            predictions.append(calculate_avg(results, targets, hand_write_results[i].detach().numpy()))

    i = 0
    c = 0
    fig = plt.figure()
    while i < batch_num:
        for j in range(CNN.BATCH_SIZE_TEST):
            if c >= len(hand_write_results):
                i = batch_num
                break
            plt.subplot(3, 5, c + 1)
            plt.tight_layout()
            plt.imshow(imgs[i][j][0], cmap = 'gray', interpolation = 'none')
            if is_knn:
                plt.title("s{}dr{}kprediction:\n{}".format(s, dr, predictions[c]))
            else:
                plt.title("s{}dr{}prediction:\n{}".format(s, dr, predictions[c]))
            plt.xticks([])
            plt.yticks([])
    fig.show()

**prediction_by_knn**

In [ ]:
def prediction_by_knn(filename, k, greek_submodel, results, targets):
    image = read_image(filename).float()
    image = image[None, :]
    output = greek_submodel(image)
    print('\nDistance from input image to greek database:')
    prediction = knn(k, results, targets, output[0].detach().numpy())
    fig = plt.figure()
    plt.imshow(image[0][0], cmap = 'gray', interpolation = 'none')
    plt.title("Prediction: %s" % prediction)
    plt.xticks([])
    plt.yticks([])
    fig.show()

**Define main function**

In [ ]:
def main():
    # load the model
    loaded_net = CNN.NeuralNetwork()
    loaded_net_state_dict = torch.load('samruddhi_neural.pt')
    loaded_net.load_state_dict(loaded_net_state_dict)
    loaded_net.eval()

    dataset_greek_values, dataloader_greek_values = save_to_csv()

    for i in CONV_FILTER_SIZE:
        for j in DROPOUT_RATE:
            print("For kernel size = {} and drop rate = {}\n".format(i, j))

            greek_submodel = Submodel(i, j)

            print("Building embedding space of greek-1")
            results, targets = embedding(greek_submodel, dataloader_greek_values)
            print("Embedding space of greek-1 built")

            alpha, beta, gamma = get_greek_letter(results, targets)

            print("\nDistance from selected alpha to others: ")
            calculate_avg(results, targets, alpha)
            print("\nDistance from selected beta to others: ")
            calculate_avg(results, targets, beta)
            print("\nDistance from selected gamma to others: ")
            calculate_avg(results, targets, gamma)
            print('\n')

            evaluate_greek_written(greek_submodel, results, targets, False, i, j)
            evaluate_greek_written(greek_submodel, results, targets, True, i, j)

    greek_submodel = Submodel()
    results, targets = embedding(greek_submodel, dataloader_greek_values)
    prediction_by_knn(KNN_INPUT_FILENAME, 3, greek_submodel, results, targets)


if __name__ == '__main__':
    main()